In [ ]:
import sqlite3
import json
import httpx
import os

from dotenv import load_dotenv

load_dotenv()

# Uncomment project_color & project_hex to include them in each time entry

DB = "toggl_time_tracking.sqlite"

SCHEMA = """
CREATE TABLE IF NOT EXISTS time_entries (
    entry_id       INTEGER PRIMARY KEY,
    description    TEXT NOT NULL,
    project_id     INTEGER NOT NULL,
    project_name   TEXT NOT NULL,
    -- project_color  TEXT,
    -- project_hex    TEXT,
    seconds        INTEGER NOT NULL,
    start          TEXT NOT NULL,
    stop           TEXT,
    tag_ids        TEXT,
    tag_names      TEXT,
    at             TEXT NOT NULL
);
"""

def init_db():
    with sqlite3.connect(DB) as db:
        db.execute(SCHEMA)
        db.commit()

def upsert_sqlite(entry):
    with sqlite3.connect(DB) as db:
        db.execute("""
            INSERT INTO time_entries (
                entry_id,
                description,
                project_id,
                project_name,
                -- project_color,
                -- project_hex,
                seconds,
                start,
                stop,
                tag_ids,
                tag_names,
                at
            )
            --VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ON CONFLICT(entry_id) DO UPDATE SET
                description    = excluded.description,
                project_id     = excluded.project_id,
                project_name   = excluded.project_name,
                -- project_color  = excluded.project_color,
                -- project_hex    = excluded.project_hex,
                seconds        = excluded.seconds,
                start          = excluded.start,
                stop           = excluded.stop,
                tag_ids        = excluded.tag_ids,
                tag_names      = excluded.tag_names,
                at             = excluded.at;
        """, (
            entry["entry_id"],
            entry["description"],
            entry["project_id"],
            entry["project_name"],
            # entry.get("project_color"),
            # entry.get("project_hex"),
            entry["seconds"],
            entry["start"],
            entry.get("stop"),
            json.dumps(entry.get("tag_ids", [])),
            json.dumps(entry.get("tag_names", [])),
            entry["at"],
        ))


In [ ]:
init_db()

In [ ]:
url  = f"https://api.track.toggl.com/reports/api/v3/workspace/{os.environ["WORKSPACE_ID"]}/search/time_entries"
auth = (os.environ["TOGGL_TOKEN"], "api_token")
payload = {
    "start_date": "2025-01-01",
    "end_date":   "2025-06-11",
    "page_size":  100,
    "enrich_response": True,
    "grouped": True
}
data = httpx.post(url, json=payload, auth=auth).json()

i = 0
while True:
    resp = httpx.post(url, json=payload, auth=auth, timeout=30)
    resp.raise_for_status()                       # fail fast on errors
    rows = resp.json()

    for row in rows:
        meta = {
            "project_id":    row["project_id"],
            "project_name":  row.get("project_name"),
            # "project_color": row["project_color"],
            # "project_hex":   row["project_hex"],
            "description":   row["description"],
            "tag_ids":       row.get("tag_ids", []),
            "tag_names":     row.get("tag_names", []),
        }

        # time_entries now holds 1-N actual entries for this meta combo
        for time_entry in row["time_entries"]:
            flat = {
                **meta,
                "entry_id":  time_entry["id"],
                "start":     time_entry["start"],
                "stop":      time_entry["stop"],
                "seconds":   time_entry["seconds"],
                "at":        time_entry["at"],
            }
            upsert_sqlite(flat)
            print(i)
            i += 1

    # pagination – move to next page if the header is present
    nxt = resp.headers.get("X-Next-ID")
    if not nxt:
        break
    payload["first_id"] = nxt